# Spark Setup and Data Load

##Installation of Spark

In [22]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark2.4.5
!wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
# unzip it
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
# install findspark
!pip install -q findspark
# clone github repo
!git clone https://github.com/PiotrMaciejKowalski/BigData2022-films
# Przeniesienie plików z BigData2022-films do katalogu nadrzędnego
!mv BigData2022-films/* .
!mv BigData2022-films/.* .
!rmdir BigData2022-films

fatal: destination path 'BigData2022-films' already exists and is not an empty directory.
mv: cannot move 'BigData2022-films/colabs' to './colabs': Directory not empty
mv: cannot move 'BigData2022-films/docs' to './docs': Directory not empty
mv: cannot move 'BigData2022-films/lib' to './lib': Directory not empty
mv: cannot move 'BigData2022-films/notebooks' to './notebooks': Directory not empty
mv: cannot move 'BigData2022-films/reports' to './reports': Directory not empty
mv: cannot move 'BigData2022-films/stripped' to './stripped': Directory not empty
mv: cannot move 'BigData2022-films/tests' to './tests': Directory not empty
mv: cannot move 'BigData2022-films/tutorials' to './tutorials': Directory not empty
mv: cannot move 'BigData2022-films/.' to './.': Device or resource busy
mv: cannot move 'BigData2022-films/..' to './..': Device or resource busy
mv: cannot move 'BigData2022-films/.git' to './.git': Directory not empty
rmdir: failed to remove 'BigData2022-films': Directory not e

In [31]:
import os

# setup environment variables for our Spark Session to work
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = '/content/spark-3.2.1-bin-hadoop3.2'

from lib.pyspark_startup import init, load

In [25]:
spark = init()

AnalysisException: ignored

In [ ]:
!chmod +x data_source.sh
!./data_source.sh

In [ ]:
df = load(spark)

NameError: ignored

In [ ]:
df.show(5)